# Chapter 2: Emitting Basic MLIR

This is an xDSL version of the Toy compiler, as described in the [MLIR tutorial](https://mlir.llvm.org/docs/Tutorials/Toy/)

This tutorial will be illustrated with a toy language that we’ll call “Toy” (naming is hard…). Toy is a tensor-based language that allows you to define functions, perform some math computation, and print results.

In [1]:
import sys
sys.path.append('../src')

In [1]:
from xdsl import *
from xdsl.ir import *
from xdsl.irdl import *
from xdsl.dialects.func import *
from xdsl.dialects.arith import *
from xdsl.dialects.builtin import *
from xdsl.parser import *
from xdsl.printer import *
from xdsl.util import *

# MLContext, containing information about the registered dialects
context = MLContext()

# Some useful dialects
arith = Arith(context)
func = Func(context)
builtin = Builtin(context)

# Printer used to pretty-print MLIR data structures
printer = Printer(target=Printer.Target.MLIR)

@dataclass
class Toy:
    ctx: MLContext

    def __post_init__(self):
        self.ctx.register_op(ConstantOp)
        # self.ctx.register_op(Call)
        # self.ctx.register_op(Return)

TensorTypeF64 = TensorType.from_type_and_list(f64)

@irdl_op_definition
class ConstantOp(Operation):
    '''
    Constant operation turns a literal into an SSA value. The data is attached
    to the operation as an attribute. For example:

    ```mlir
      %0 = toy.constant dense<[[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]]>
                        : tensor<2x3xf64>
    ```
    '''
    name: str = "toy.constant"
    result = ResultDef(AnyAttr()) #TensorType[F64])
    value = AttributeDef(AnyAttr()) #F64ElementsAttr

    # TODO verify that the result and value type are equal

    @staticmethod
    def from_list(data: List[float], shape: List[int] = []):
        value = DenseIntOrFPElementsAttr.tensor_from_list(data, f64, shape)
        return ConstantOp.create(result_types=[value.type], attributes={"value": value})

    # def print(self, printer: Printer):
    #     printer.print_attribute(self.attributes['value'])
        # printer.print(' bla')

const_op = ConstantOp.from_list([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], [2, 3])
printer.print_op(const_op)

%0 = "toy.constant"() {"value" = dense<[

Exception: unexpected attribute type in DenseIntOrFPElementsAttr: <class 'xdsl.dialects.builtin.FloatAttr'>

In [3]:
my_int = IntAttr.from_int(42)
printer.print_attribute(my_int)

!int<42>

In [4]:
from pathlib import Path

ast_toy = Path() / 'toy' / 'ast.toy'

In [5]:
with open(ast_toy, 'r') as f:
    print(f.read())

# RUN: toyc-ch1 %s -emit=ast 2>&1 | FileCheck %s

# User defined generic function that operates on unknown shaped arguments.
def multiply_transpose(a, b) {
  return transpose(a) * transpose(b);
}

def main() {
  # Define a variable `a` with shape <2, 3>, initialized with the literal value.
  # The shape is inferred from the supplied literal.
  var a = [[1, 2, 3], [4, 5, 6]];
  # b is identical to a, the literal array is implicitly reshaped: defining new
  # variables is the way to reshape arrays (element count in literal must match
  # the size of specified shape).
  var b<2, 3> = [1, 2, 3, 4, 5, 6];

  # This call will specialize `multiply_transpose` with <2, 3> for both
  # arguments and deduce a return type of <2, 2> in initialization of `c`.
  var c = multiply_transpose(a, b);
  # A second call to `multiply_transpose` with <2, 3> for both arguments will
  # reuse the previously specialized and inferred version and return `<2, 2>`
  var d = multiply_transpose(b, a);
  # A new call w

In [6]:
from toy.Parser import Parser

with open(ast_toy, 'r') as f:
    parser = Parser(ast_toy, f.read())

print(parser.parseModule().dump())

Module:
  Function 
    Proto 'multiply_transpose' toy/ast.toy:4:1'
    Params: [a, b]
    Block {
      Return
        BinOp: * toy/ast.toy:5:25
          Call 'transpose' [ toy/ast.toy:5:10
            var: a toy/ast.toy:5:20
          ]
          Call 'transpose' [ toy/ast.toy:5:25
            var: b toy/ast.toy:5:35
          ]
    } // Block
  Function 
    Proto 'main' toy/ast.toy:8:1'
    Params: []
    Block {
      VarDecl a<> toy/ast.toy:11:3
        Literal: <2, 3>[ <3>[ 1.000000e+00, 2.000000e+00, 3.000000e+00], <3>[ 4.000000e+00, 5.000000e+00, 6.000000e+00]] toy/ast.toy:11:11
      VarDecl b<2, 3> toy/ast.toy:15:3
        Literal: <6>[ 1.000000e+00, 2.000000e+00, 3.000000e+00, 4.000000e+00, 5.000000e+00, 6.000000e+00] toy/ast.toy:15:17
      VarDecl c<> toy/ast.toy:19:3
        Call 'multiply_transpose' [ toy/ast.toy:19:11
          var: a toy/ast.toy:19:30
          var: b toy/ast.toy:19:33
        ]
      VarDecl d<> toy/ast.toy:22:3
        Call 'multiply_transpose' [ t